In [102]:
import numpy as np
import math
from dataclasses import dataclass

In [ ]:
from dataclasses import dataclass
from typing import List, Tuple
import numpy as np

@dataclass
class CVRPInstance:
    """
    Class to represent a Capacitated Vehicle Routing Problem (CVRP) instance.
    
    Attributes
    ----------
    n_customers : int
        Number of customers in the instance.
    capacity : int
        Capacity of each vehicle.
    demands : List[int]
        List of demands for each customer.
    coordinates : List[Tuple[float, float]]
        List of (x, y) coordinates for each customer.
    dist : np.ndarray
        Matrix of distances between customers.
    """
    n_customers: int
    capacity: int
    demands: List[int]
    # coordinates: List[Tuple[float, float]]
    dist: np.ndarray

@dataclass
class CVRPSolution:
    """
    Class to represent a Capacitated Vehicle Routing Problem (CVRP) solution.
    
    Attributes
    ----------
    routes : List[int]
        List of routes, where each route is represented as a list of customer indices.
    n_vehicles: int
        Number of vehicles used in the solution.
    cost: int
        Total cost of the solution.
    """
    routes: List[int]
    n_vehicles: int
    cost: int

def read_cvrp_instance(filename: str) -> CVRPInstance:
    """
    Reads a CVRP instance from a file.
    
    Parameters
    ----------
    filename : str
        Path to the CVRP instance file.
    
    Returns
    -------
    CVRPInstance
        An instance of the CVRPInstance class populated with data from the file.
    """
    with open(filename, 'r', encoding='utf-8') as f:
        lines = [line.strip() for line in f]
    
    # Get number of customers and it's capacity
    n_customers = int(lines[3].split('\t')[1]) - 1
    vehicle_capacity = int(lines[5].split('\t')[1])

    # Get depot and customer coordinates
    coord = []
    for i in range(7, 7 + n_customers + 1):
        x, y = map(int, lines[i].split('\t')[1:])
        coord.append((x, y))

    # Get customer demand
    demands = []
    for j in range(i+2, i+2 + n_customers + 1):
        demand = int(lines[j].split('\t')[1])
        demands.append(demand)

    # Calculate distance matrix
    costumers_dist = np.zeros((n_customers+1, n_customers+1))
    for i in range(n_customers+1):
        for j in range(n_customers+1):
            if i != j:
                dist = math.sqrt((coord[i][0] - coord[j][0])**2 + (coord[i][1] - coord[j][1])**2)
                costumers_dist[i][j] = math.floor(dist + 0.5)
            else:
                costumers_dist[i][j] = 0
        
    return CVRPInstance(n_customers=n_customers, capacity=vehicle_capacity, demands=demands, dist=costumers_dist)

def evaluate_solution(instance: CVRPInstance, final_routes: List[List[int]]) -> CVRPSolution:
    """
    Evaluates a CVRP solution by calculating its total cost.
    
    Parameters
    ----------
    instance : CVRPInstance
        The CVRP instance for which the solution is evaluated.
    final_routes : List[List[int]]
        The list of routes in the solution, where each route is represented as a list of customer indices.
    
    Returns
    -------
    CVRPSolution
        An instance of the CVRPSolution class containing the routes, number of vehicles, and total cost.
    """
    # Calculate total cost and return solution
    total_cost = 0
    for route in final_routes:
        # depot -> first customer
        cost = instance.dist[0][route[0]]

        # Customer -> Customer
        for i in range(len(route)-1):
            cost += instance.dist[route[i]][route[i+1]]

        # last customer -> depot
        cost += instance.dist[route[-1]][0]

        # Add to total cost
        total_cost += cost

    return CVRPSolution(routes=final_routes, n_vehicles=len(final_routes), cost=total_cost)      

In [134]:
def savings_heuristic(instance: CVRPInstance) -> CVRPSolution:
    # Initialize variables
    n = instance.n_customers 
    routes = [[i] for i in range(1, n+1)]  # Insert each customer in its own route
    route_demands = [instance.demands[i] for i in range(1, n+1)]
    route_start_end = [(i, i) for i in range(1, n+1)]  # (start, end) of each route

    # Calculate savings
    savings = []
    for i in range(1, n+1):
        for j in range(i+1, n+1):
            s = instance.dist[0][i] + instance.dist[0][j] - instance.dist[i][j]
            savings.append((s, i, j))
    savings.sort(reverse=True)

    # Mapping from costumer to its route index
    client_route = {i: idx for idx, i in enumerate(range(1, n+1))}

    for s, i, j in savings:
        ri = client_route[i]
        rj = client_route[j]
        if ri == rj:
            continue  # Clients already in the same route

        # Check if i is at the end of its route and j is at the start of its route
        if route_start_end[ri][1] == i and route_start_end[rj][0] == j:
            # If they are, check vehicle capacity
            if route_demands[ri] + route_demands[rj] <= instance.capacity:
                # Join the routes
                routes[ri] = routes[ri] + routes[rj]
                route_demands[ri] += route_demands[rj] # Sum demands from both routes
                route_start_end[ri] = (route_start_end[ri][0], route_start_end[rj][1]) # Fix route start and end

                # Update costumer -> route mapping
                for client in routes[rj]:
                    client_route[client] = ri
                    
                # Remove route rj
                routes[rj] = []
                route_demands[rj] = 0
                route_start_end[rj] = (None, None)

    # Remove empty routes
    final_routes = [r for r in routes if r]

    # Evaluate final solution
    solution = evaluate_solution(instance, final_routes)
        
    return solution

    
def regret_insertion_heuristic(instance: CVRPInstance, lambda_param=1):
    # Initialize variables
    n = instance.n_customers
    unrouted = set(range(1, n+1))
    routes = []

    # Initialize routes with farthest customers from the depot (avoiding long routes)
    min_needed_routes = math.ceil(sum(instance.demands[1:]) / instance.capacity) # Demands / Capacity
    reps = sorted(unrouted, key=lambda i: instance.dist[0][i], reverse=True)[:min_needed_routes]
    for r in reps:
        routes.append([r])
        unrouted.remove(r)

    # Insert remaining customers using regret criterion
    while unrouted:
        best_regret = -float('inf')
        best_client = None
        best_route = None
        best_pos = None

        # For every unrouted customer
        for k in unrouted:
            insertion_costs = []
            for ridx, route in enumerate(routes):
                # Calculate insertion cost between two customers
                for pos in range(len(route)+1):
                    prev = 0 if pos == 0 else route[pos-1]
                    nxt = 0 if pos == len(route) else route[pos]
                    cost = instance.dist[prev][k] + instance.dist[k][nxt] - lambda_param * instance.dist[prev][nxt]
                    
                    # Checks for insertion feasibility
                    route_demand = sum(instance.demands[c] for c in route)
                    if route_demand + instance.demands[k] <= instance.capacity:
                        insertion_costs.append((cost, ridx, pos))
            
            # Calculate regret value if more than one insertion is possible
            if len(insertion_costs) >= 2:
                insertion_costs.sort()
                regret = insertion_costs[1][0] - insertion_costs[0][0]
                if regret > best_regret:
                    best_regret = regret
                    best_client = k
                    best_route = insertion_costs[0][1]
                    best_pos = insertion_costs[0][2]
            
            # If only one insertion is possible, regret is zero
            elif len(insertion_costs) == 1:
                regret = 0
                if regret > best_regret:
                    best_regret = regret
                    best_client = k
                    best_route = insertion_costs[0][1]
                    best_pos = insertion_costs[0][2]

        # Insert the client with the highest regret in the best position
        if best_client is not None:
            routes[best_route].insert(best_pos, best_client)
            unrouted.remove(best_client)
        else:
            # No possible insertion: creates a new route
            for k in unrouted:
                if instance.demands[k] <= instance.capacity:
                    routes.append([k])
                    unrouted.remove(k)
                    break
                
    # Evaluate final solution
    solution = evaluate_solution(instance, routes)
        
    return solution

In [109]:
instances = []

for i in range(1, 9):
    instance = read_cvrp_instance(f'instances/instance{i}.vrp')
    instances.append(instance)

In [110]:
savings_results = []

for instance in instances:
    solution = savings_heuristic(instance)
    savings_results.append(solution)

In [140]:
regret_insertion_results = []

for instance in instances:
    print("Solving instance with regret insertion heuristic with {} costumers".format(instance.n_customers))
    solution = regret_insertion_heuristic(instance)
    regret_insertion_results.append(solution)

Solving instance with regret insertion heuristic with 100 costumers
Solving instance with regret insertion heuristic with 203 costumers
Solving instance with regret insertion heuristic with 302 costumers
Solving instance with regret insertion heuristic with 400 costumers
Solving instance with regret insertion heuristic with 501 costumers
Solving instance with regret insertion heuristic with 612 costumers
Solving instance with regret insertion heuristic with 700 costumers
Solving instance with regret insertion heuristic with 800 costumers
